In [1]:
! pip install spacy

In [2]:
!git clone https://github.com/abecode/teaching_reviews.git

Cloning into 'teaching_reviews'...
remote: Enumerating objects: 230, done.
remote: Counting objects: 100% (230/230), done.
remote: Compressing objects: 100% (198/198), done.
remote: Total 230 (delta 116), reused 123 (delta 31), pack-reused 0 (from 0)
Receiving objects: 100% (230/230), 3.57 MiB | 3.68 MiB/s, done.
Resolving deltas: 100% (116/116), done.


In [3]:
!ls -lrt teaching_reviews/data_jsonl/

total 42356
-rw-r--r-- 1 root root  5488435 Apr 22 08:20 teaching_reviews_pilot1_spans_20250401.json
-rw-r--r-- 1 root root 37667169 Apr 22 08:20 teaching_reviews_pilot1_spans_reviews_20250414.json
-rw-r--r-- 1 root root   131264 Apr 22 08:20 teaching_reviews_sentences.jsonl
-rw-r--r-- 1 root root    75336 Apr 22 08:20 teaching_reviews_whole_review.jsonl


In [4]:
cat teaching_reviews/data_jsonl/teaching_reviews_pilot1_spans_20250401.json | head

{"text":"Abe is a practical, passionate, and thoughtful teacher who knows how to balance between making sure that students are learning the material and getting fair grades. My only feedback is that the class seems to be covering a lot of materials, 3 hours and 15 mins don't seem enough to cover it all, at some point by the end of each class most students check out and it became just a matter of being there, maybe a better distribution of class activity will be helpful, more peer interaction and cut back on some of the material or demand that student read three pages which covers the basics of each day's class before they come to the class.","meta":{"filename":"202020SEIS632-03.txt","linenum":0},"_input_hash":33684866,"_task_hash":-808106093,"tokens":[{"text":"Abe","start":0,"end":3,"id":0,"ws":true},{"text":"is","start":4,"end":6,"id":1,"ws":true},{"text":"a","start":7,"end":8,"id":2,"ws":true},{"text":"practical","start":9,"end":18,"id":3,"ws":false},{"text":",","start":18,"end":19,"

In [5]:
import json
from pathlib import Path
from collections import OrderedDict, defaultdict


import spacy
from spacy.tokens import Doc, Span
from spacy import displacy

# Load your model — should match the one you used in Prodigy
nlp = spacy.load("en_core_web_sm")  # or your custom model

# Load Prodigy data
examples = [json.loads(line) for line in Path("teaching_reviews/data_jsonl/teaching_reviews_pilot1_spans_20250401.json").read_text().splitlines()]

# Create an ordered default dict with key being the prodigy input hash
# and the value being a dictionary whose key is the annotator/session id
# and the value is the annotation
class OrderedDefaultDict(OrderedDict):
    def __init__(self, default_factory=None, *args, **kwargs):
        #in python3 you can omit the args to super
        super(OrderedDefaultDict, self).__init__(*args, **kwargs)
        self.default_factory = default_factory
    def __missing__(self, key):
        if self.default_factory is None:
            raise KeyError(key)
        self[key] = value = self.default_factory()
        return value

docs = OrderedDefaultDict(dict)
annotators = set()

for eg in examples:
    input_hash = eg["_input_hash"]
    annotator = eg["_annotator_id"]
    doc = nlp.make_doc(eg["text"])
    spans = []
    for span in eg.get("spans", []):
        start = span["start"]
        end = span["end"]
        label = span["label"]
        span_obj = doc.char_span(start, end, label=label, alignment_mode="contract")
        if span_obj:
            spans.append(span_obj)
    doc.spans["sc"] = spans  # store under key "sc" for spancat
    annotators.add(annotator)
    docs[input_hash][annotator] = doc

In [6]:
#examples[0:2]
len(docs.keys())

192

In [7]:
# Count total number of tokens (one doc per input_hash, ignoring annotator duplicates)
total_tokens = 0

for annotator_docs in docs.values():
    doc = next(iter(annotator_docs.values()))  # just pick one annotator's doc
    total_tokens += len(doc)

print(f"Total number of tokens: {total_tokens}")


Total number of tokens: 13061


In [8]:
for input_hash in docs.keys():
    for annotator in docs[input_hash]:
        print(input_hash, annotator)
        #print(docs[input_hash][annotator])
        displacy.render(docs[input_hash][annotator], style="span", options={"spans_key": "sc"}, jupyter=True)
        print()
    break

33684866 pilot1_spans-abe



33684866 pilot1_spans-%3Cyour%20name%3E%20%20(replace%20%3Cyour%20name%3E%20with%20your%20first%20name



33684866 pilot1_spans-%3CJenny%3E



33684866 pilot1_spans-Henry



33684866 pilot1_spans-%3CMengyuan%20Cui%3E



33684866 pilot1_spans-meenu



33684866 pilot1_spans-rohith


### Autoencoder

# try ipytree

In [ ]:
# !pip install ipytree
# from ipytree import Tree, Node


In [ ]:
# from google.colab import output
# output.enable_custom_widget_manager()

In [ ]:
# tree = Tree()
# for input_hash in docs.keys():
#     node_1st = Node(str(input_hash))
#     tree.add_node(node_1st)
#     for annotator in docs[input_hash]:
#         node_2nd = Node(str(input_hash) + "_" + annotator)
#         html = displacy.render(docs[input_hash][annotator], style="span", options={"spans_key": "sc"}, jupyter=False)
#         node_3rd = Node(html)
#         node_1st.add_node(node_2nd)
#         node_2nd.add_node(node_3rd)
# tree


this didn't look so great

# try streamlit

In [9]:
!pip install -q streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.8/9.8 MB 96.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 121.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 8.4 MB/s eta 0:00:00


In [10]:
%%writefile app.py

import streamlit as st

st.write('Hello, *World!* :sunglasses:')

Writing app.py


In [11]:
!npm install localtunnel

⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙
added 22 packages in 4s
⠙
⠙3 packages are looking for funding
⠙  run `npm fund` for details
⠙

In [ ]:
# !streamlit run app.py &>/content/logs.txt &
# maybe better to run this in the terminal w/o the ampersand/background task

In [12]:
import urllib
print("Password/Enpoint IP for localtunnel is:",urllib.request.urlopen('https://ipv4.icanhazip.com').read().decode('utf8').strip("\n"))

Password/Enpoint IP for localtunnel is: 34.125.63.34


In [ ]:
#! curl https://loca.lt/mytunnelpassword

In [ ]:
#!npx localtunnel --port 8501

come back to this

# try pandas

In [13]:
from IPython.display import HTML, display
import pandas as pd

# style all pandas data tables
display(HTML("""
<style>
    .dataframe td {
        max-width: 1800px;
        white-space: normal !important;
        word-wrap: break-word;
    }
</style>
"""))

# convert docs to a pandas dataframe
#df = pd.DataFrame(columns=["input_hash", "annotator", "text"])
df_list = [] # Create an empty list to store data for each row
for input_hash in docs.keys():
    for annotator in docs[input_hash]:
        df_list.append(
            {"input_hash": input_hash,
             "annotator": annotator,
             "text": docs[input_hash][annotator].text,
             "spans": displacy.render(
                 docs[input_hash][annotator],
                 style="span",
                 options={"spans_key": "sc"},
                 jupyter=False
                 )
             }
            )

    break



# Create the DataFrame outside the loop using pd.DataFrame.from_records
df = pd.DataFrame.from_records(df_list)
#df
def remove_line_breaks(text):
    return text.replace('\n', '')
def clean_html(html):
    return html.replace('\n', '').replace('\r', '').strip()
def clean_annotator(text):
    return text.replace('pilot1_spans-', '').replace('\r', '').strip()


#df_styled = df.style.format({'spans': remove_line_breaks})
#df_styled = df.style.format({'spans': clean_html})

#display(HTML(df_styled.to_html(escape=False)));

# Add a scrollable div around the styled DataFrame
styled_html = df.style.format({'spans': clean_html,
                              'annotator': clean_annotator}).to_html(escape=False)
scroll_html = f"""
<div style="overflow-x: auto; max-width: 100%; border: 1px solid #ddd; padding: 10px;">
  <style>
    /* Make annotator column narrower */
    th.col1, td.col1 {{
        width: 150px;
        max-width: 150px;
        white-space: nowrap;
    }}
    /* Optional: style other columns */
    th.col3, td.col3 {{
        min-width: 600px;
    }}
  </style>
  {styled_html}
</div>
"""
display(HTML(scroll_html))



In [14]:
data = {'Text': ["Hello <br> World", "Hello \n World"]}
df = pd.DataFrame(data)

def add_line_breaks(text):
    return text.replace('\n', '<br>')

df_styled = df.style.format({'Text': add_line_breaks})

html_table = df_styled.to_html()

#with open('output.html', 'w') as file:
#    file.write(html_table)
HTML(html_table)

,Text
0,Hello World
1,Hello World


### Train AutoEncoder

In [15]:
import spacy
import json
from pathlib import Path
from collections import OrderedDict

# Load your SpaCy model
nlp = spacy.load("en_core_web_sm")

# Load Prodigy annotations
examples = [json.loads(line) for line in Path("teaching_reviews/data_jsonl/teaching_reviews_pilot1_spans_20250401.json").read_text().splitlines()]

# Process text and spans
docs = OrderedDict()

for eg in examples:
    input_hash = eg["_input_hash"]
    annotator = eg["_annotator_id"]
    doc = nlp.make_doc(eg["text"])
    spans = []
    for span in eg.get("spans", []):
        start = span["start"]
        end = span["end"]
        label = span["label"]
        span_obj = doc.char_span(start, end, label=label, alignment_mode="contract")
        if span_obj:
            spans.append(span_obj)
    doc.spans["sc"] = spans  # Store under key "sc" for Prodigy-style spans
    docs[input_hash] = {"annotator": annotator, "doc": doc}


In [16]:
train_texts = [eg["text"] for eg in examples]

In [17]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer, Trainer, TrainingArguments
import torch

# Load pre-trained GPT-2 model and tokenizer
model = GPT2LMHeadModel.from_pretrained("gpt2")
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

# Add padding token to tokenizer (use eos_token or create a new pad token)
tokenizer.pad_token = tokenizer.eos_token  # or you can add a custom pad token like '[PAD]'

# Tokenize the text data for training
inputs = tokenizer(train_texts, padding=True, truncation=True, return_tensors="pt", max_length=512)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [18]:
class TextDataset(torch.utils.data.Dataset):
    def __init__(self, inputs):
        self.inputs = inputs

    def __len__(self):
        return len(self.inputs["input_ids"])

    def __getitem__(self, idx):
        # Ensure labels are same as input_ids for language modeling
        return {
            "input_ids": torch.tensor(self.inputs["input_ids"][idx]),
            "attention_mask": torch.tensor(self.inputs["attention_mask"][idx]),
            "labels": torch.tensor(self.inputs["input_ids"][idx]),  # labels = input_ids
        }

train_dataset = TextDataset(inputs)

In [20]:
# Training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=2,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=8,
    logging_dir='./logs',
    logging_steps=10,
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
)


In [21]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: fennialee608 (fennialee608-university-of-st-thomas) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


<ipython-input-18-13ed050748ea>:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  "input_ids": torch.tensor(self.inputs["input_ids"][idx]),
<ipython-input-18-13ed050748ea>:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  "attention_mask": torch.tensor(self.inputs["attention_mask"][idx]),
<ipython-input-18-13ed050748ea>:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  "labels": torch.tensor(self.inputs["input_ids"][idx]),  # labels = input_ids
`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausa

Step,Training Loss
10,2.181400
20,0.574000
30,0.452100
40,0.540100
50,0.445100
60,0.506400


TrainOutput(global_step=68, training_loss=0.7435249545994926, metrics={'train_runtime': 319.8601, 'train_samples_per_second': 6.909, 'train_steps_per_second': 0.213, 'total_flos': 569616629760000.0, 'train_loss': 0.7435249545994926, 'epoch': 1.9819494584837545})

In [40]:
model.eval()

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D(nf=2304, nx=768)
          (c_proj): Conv1D(nf=768, nx=768)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=3072, nx=768)
          (c_proj): Conv1D(nf=768, nx=3072)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [43]:
from transformers import GPT2Tokenizer, GPT2Model, GPT2LMHeadModel
import torch

# Load the pre-trained GPT-2 model and tokenizer
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model = GPT2Model.from_pretrained("gpt2")
model_with_lm_head = GPT2LMHeadModel.from_pretrained("gpt2")

# Set padding token to eos_token (end-of-sequence token)
tokenizer.pad_token = tokenizer.eos_token

augmented_examples = []


In [48]:
# Iterate  docs dictionary containing the annotated data
for input_hash, annotator_docs in docs.items():
    for annotator, doc in annotator_docs.items():
        # Ensure doc is a string (convert if necessary)
        text = str(doc)

        # Encode the text using GPT-2 tokenizer
        inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
        outputs = model(**inputs)
        hidden_states = outputs.last_hidden_state

        # Manipulate the latent space (for example, add random noise)
        augmented_hidden_states = hidden_states + torch.randn_like(hidden_states) * 0.1  # Example noise

        # Decode the augmented latent space to generate new text
        generated_ids = model_with_lm_head.generate(inputs["input_ids"], max_length=1024)
        augmented_text = tokenizer.decode(generated_ids[0], skip_special_tokens=True)

        # Re-create the doc and spans (you may need to adjust spans for new text)
        augmented_doc = nlp.make_doc(augmented_text)

        # Save the augmented example with the new text (you can adjust spans if needed)
        augmented_examples.append({
            "text": augmented_text,
            "spans": []  # Adjust spans here if necessary
        })


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generati

KeyboardInterrupt: 